# [ml4ir](https://github.com/salesforce/ml4ir) 
#### open source, modular, python3, tensorflow2.0 library for IR based ML applications
--------------------

![image.png](images/ml4ir.png)

### First, let's load the data and take a look at it

In [6]:
from ml4ir.base.io import file_io
import glob
import logging
import pandas as pd
import os

# Pandas options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Setup logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("Logger is initialized...")


# Load data
CSV_DATA_DIR = '../ml4ir/applications/ranking/tests/data/csv'

df = file_io.read_df_list(glob.glob(os.path.join(CSV_DATA_DIR, "train", "*.csv")), log=logger)

logger.info(df.shape)

df[[c for c in df.columns if c != "clicked"] + ["clicked"]].sort_values(["query_id", "rank"]).head(7)

DEBUG:root:Logger is initialized...
INFO:root:Reading 1 files from [../ml4ir/applications/ranking/tests/data/csv/train/file_0.csv, ..
INFO:root:(5676, 10)


,query_id,query_text,rank,text_match_score,page_views_score,quality_score,domain_id,domain_name,name_match,clicked
4365,query_0,UQHA3QP4ZVO,1,1.101297,0.002044,0.00000,0,domain_0,0,1
4366,query_0,UQHA3QP4ZVO,2,0.380570,0.004078,0.30103,0,domain_0,0,0
4371,query_1,8M3NWYX4E6I,1,1.024334,0.008686,0.30103,1,domain_1,1,1
4368,query_1,8M3NWYX4E6I,2,0.821515,0.200264,0.00000,1,domain_1,1,0
4370,query_1,8M3NWYX4E6I,3,0.821323,0.200264,0.00000,1,domain_1,0,0
4369,query_1,8M3NWYX4E6I,4,0.821515,0.200264,0.00000,1,domain_1,0,0
4367,query_1,8M3NWYX4E6I,5,0.821323,0.200264,0.00000,1,domain_1,1,0


### Let's define the feature configuration for our data

### ... brace yourselves!

In [40]:
# Set up the feature configurations
from ml4ir.base.features.feature_config import parse_config
from ml4ir.base.features.feature_config import ExampleFeatureConfig
from ml4ir.base.config.keys import TFRecordTypeKey
import json

feature_config_yaml = '''
query_key: 
  name: query_id
  node_name: query_id
  trainable: false
  dtype: string
  log_at_inference: true
  feature_layer_info:
    type: numeric
    shape: null
  serving_info:
    name: queryId
    required: false
    default_value: ""
  tfrecord_type: context
label:
  name: clicked
  node_name: clicked
  trainable: false
  dtype: int64
  log_at_inference: true
  feature_layer_info:
    type: numeric
    shape: null
  serving_info:
    name: clicked
    required: false
    default_value: 0
  tfrecord_type: sequence
features:
  - name: rank
    node_name: rank
    trainable: false
    dtype: int64
    log_at_inference: true
    feature_layer_info:
      type: numeric
      shape: null
    serving_info:
      name: originalRank
      required: true
      default_value: 0
    tfrecord_type: sequence
  - name: text_match_score
    node_name: text_match_score
    trainable: true
    dtype: float
    log_at_inference: false
    feature_layer_info:
      type: numeric
      shape: null
    serving_info:
      name: textMatchScore
      required: true
      default_value: 0.0
    tfrecord_type: sequence
  - name: page_views_score
    node_name: page_views_score
    trainable: true
    dtype: float
    log_at_inference: false
    feature_layer_info:
      type: numeric
      shape: null
    serving_info:
      name: pageViewsScore
      required: true
      default_value: 0.0
    tfrecord_type: sequence
  - name: quality_score
    node_name: quality_score
    trainable: true
    dtype: float
    log_at_inference: false
    feature_layer_info:
      type: numeric
      shape: null
    preprocessing_info:
      - fn: signed_log
        args:
          shift: 1
    serving_info:
      name: qualityScore
      required: true
      default_value: 0.0
    tfrecord_type: sequence
  - name: name_match
    node_name: name_match
    trainable: false
    dtype: float
    log_at_inference: true
    is_secondary_label: true
    feature_layer_info:
      type: numeric
      shape: null
    serving_info:
      name: nameMatch
      required: true
      default_value: 0.0
    tfrecord_type: sequence
  - name: query_text
    node_name: query_text
    trainable: true
    dtype: string
    log_at_inference: true
    feature_layer_info:
      type: numeric
      shape: null
      fn: bytes_sequence_to_encoding_bilstm
      args:
        encoding_type: bilstm
        encoding_size: 128
        embedding_size: 128
        max_length: 20
    preprocessing_info:
      - fn: preprocess_text
        args:
          remove_punctuation: true
          to_lower: true
    serving_info:
      name: q
      required: true
      default_value: ""
    tfrecord_type: context
  - name: domain_id
    node_name: domain_id
    trainable: true
    dtype: int64
    log_at_inference: false
    is_group_metric_key: true
    feature_layer_info:
      type: numeric
      shape: null
      fn: custom_categorical_embedding
      args:
        num_buckets: 8
        embedding_size: 64
        default_value: null
    serving_info:
      name: domainID
      required: true
      default_value: 0
    tfrecord_type: context
  - name: domain_name
    node_name: domain_name
    trainable: true
    dtype: string
    log_at_inference: true
    is_group_metric_key: true
    feature_layer_info:
      type: numeric
      shape: null
      # fn: categorical_embedding_with_hash_buckets
      # args:
      #   num_hash_buckets: 4
      #   hash_bucket_size: 64
      #   embedding_size: 32
      #   merge_mode: concat
      fn: categorical_embedding_with_vocabulary_file
      args:
        vocabulary_file: '../ml4ir/applications/ranking/tests/data/config/group_name_vocab_no_id.csv'
        embedding_size: 64
        default_value: -1
        num_oov_buckets: 1
    serving_info:
      name: domainName
      required: true
      default_value: ""
    tfrecord_type: context
'''
feature_config: ExampleFeatureConfig = parse_config(TFRecordTypeKey.EXAMPLE, feature_config_yaml, logger=logger)
    
logging.info(json.dumps(feature_config.get_all_features(), indent=4))

INFO:root:Reading feature config from YAML string
INFO:root:Feature config loaded successfully
INFO:root:Trainable Features : 
text_match_score
page_views_score
quality_score
query_text
domain_id
domain_name
INFO:root:Label : clicked
INFO:root:Metadata Features : 
query_id
clicked
rank
name_match
INFO:root:[
    {
        "name": "query_id",
        "node_name": "query_id",
        "trainable": false,
        "dtype": "string",
        "log_at_inference": true,
        "feature_layer_info": {
            "type": "numeric",
            "shape": null
        },
        "serving_info": {
            "name": "queryId",
            "required": false,
            "default_value": ""
        },
        "tfrecord_type": "context"
    },
    {
        "name": "clicked",
        "node_name": "clicked",
        "trainable": false,
        "dtype": "int64",
        "log_at_inference": true,
        "feature_layer_info": {
            "type": "numeric",
            "shape": null
        },
        

## TFRecords - Examples vs SequenceExamples

![image.png](images/tfrecords.png)

### Time to load the data and save awesome TFRecords

In [41]:
from ml4ir.base.io import file_io
from ml4ir.base.data import tfrecord_writer
import glob
import os

# Load data
df = file_io.read_df_list(glob.glob(os.path.join(CSV_DATA_DIR, "train", "*.csv")))

# Save as TFRecord SequenceExample/Example
TFRECORD_DIR = '../data/pointwise_ranking_demo/'
if not os.path.exists(TFRECORD_DIR):
    os.makedirs('../data/pointwise_ranking_demo/')
tfrecord_writer.write_from_df(df,
                              tfrecord_file=os.path.join(TFRECORD_DIR, 'file_0.tfrecord'),
                              feature_config=feature_config,
                              tfrecord_type=TFRecordTypeKey.EXAMPLE)

# Let's see what it looks like
df.head()

,query_id,query_text,rank,text_match_score,page_views_score,quality_score,clicked,domain_id,domain_name,name_match
0,query_2,MHS7A7RJB1Y4BJT,2,0.473730,0.000000,0.00000,0,2,domain_2,1
1,query_2,MHS7A7RJB1Y4BJT,1,1.063190,0.205381,0.30103,1,2,domain_2,1
2,query_5,KNJNWV,6,1.368108,0.030636,0.00000,0,0,domain_0,0
3,query_5,KNJNWV,3,1.370628,0.041261,0.30103,0,0,domain_0,0
4,query_5,KNJNWV,4,1.366700,0.082535,0.30103,0,0,domain_0,0


### Load TFRecords and add custom preprocessing functions

In [64]:
from ml4ir.base.data import tfrecord_reader
from tensorflow import print as tfprint
import tensorflow as tf

@tf.function
def strip_numbers(feature_tensor):
    return tf.strings.regex_replace(feature_tensor, "[0-9]", "")

@tf.function
def signed_log(feature_tensor, shift=1.):
    """Signed log"""
    return tf.math.log(
                    tf.add(feature_tensor,
                    tf.cast(tf.constant(shift),
                            tf.float32)
                          )
                )


# Define per instance preprocessing functions
preprocessing_fns = {
    "strip_numbers": strip_numbers,
    "signed_log": signed_log
}

# Create a TFRecord dataset
dataset = tfrecord_reader.read(data_dir=TFRECORD_DIR,
                               feature_config=feature_config,
                               tfrecord_type=TFRecordTypeKey.EXAMPLE,
                               preprocessing_keys_to_fns=preprocessing_fns)

tfprint(next(iter(dataset.batch(5))))

{'query_id': FixedLenFeature(shape=[], dtype='string', default_value=''), 'clicked': FixedLenFeature(shape=[], dtype='int64', default_value=0), 'rank': FixedLenFeature(shape=[], dtype='int64', default_value=0), 'text_match_score': FixedLenFeature(shape=[], dtype='float', default_value=0.0), 'page_views_score': FixedLenFeature(shape=[], dtype='float', default_value=0.0), 'quality_score': FixedLenFeature(shape=[], dtype='float', default_value=0.0), 'name_match': FixedLenFeature(shape=[], dtype='float', default_value=0.0), 'query_text': FixedLenFeature(shape=[], dtype='string', default_value=''), 'domain_id': FixedLenFeature(shape=[], dtype='int64', default_value=0), 'domain_name': FixedLenFeature(shape=[], dtype='string', default_value='')}
({'domain_id': [[2]
 [2]
 [0]
 [0]
 [0]],
  'domain_name': [["domain_2"]
 ["domain_2"]
 ["domain_0"]
 ["domain_0"]
 ["domain_0"]],
  'name_match': [[1]
 [1]
 [0]
 [0]
 [0]],
  'page_views_score': [[0]
 [0.205380633]
 [0.0306360275]
 [0.0412614979]
 [0

### Map, Filter, Filter, Batch the Dataset

In [65]:
# Variety of map, reduce, filter, shuffle operations can be used here
# dataset = dataset.<map, filter, reduce>(tf_preprocess_fn)

# NOTE: This is lazy batching
dataset = dataset.batch(batch_size=128, drop_remainder=True)

### Or... you can do all of that for train, val and test in _one_ step!

In [66]:
from ml4ir.base.data.relevance_dataset import RelevanceDataset
from ml4ir.base.config.keys import DataFormatKey

relevance_dataset = RelevanceDataset(
        data_dir=CSV_DATA_DIR,
        data_format=DataFormatKey.CSV,
        feature_config=feature_config,
        tfrecord_type=TFRecordTypeKey.EXAMPLE,
        batch_size=128,
        preprocessing_keys_to_fns=preprocessing_fns,
        logger=logger
    )

tfprint(relevance_dataset.train)
tfprint(relevance_dataset.validation)
tfprint(relevance_dataset.test)

INFO:root:Reading 1 files from [../ml4ir/applications/ranking/tests/data/csv/train/file_0.csv, ..
INFO:root:Writing SequenceExample protobufs to : ../ml4ir/applications/ranking/tests/data/csv/tfrecord/train/file_0.tfrecord
INFO:root:Created TFRecordDataset from SequenceExample protobufs from 1 files : ['../ml4ir/applications/ranking/tests/data/csv/tfr
INFO:root:Reading 1 files from [../ml4ir/applications/ranking/tests/data/csv/validation/file_0.csv, ..
INFO:root:Writing SequenceExample protobufs to : ../ml4ir/applications/ranking/tests/data/csv/tfrecord/validation/file_0.tfrecord


{'query_id': FixedLenFeature(shape=[], dtype='string', default_value=''), 'clicked': FixedLenFeature(shape=[], dtype='int64', default_value=0), 'rank': FixedLenFeature(shape=[], dtype='int64', default_value=0), 'text_match_score': FixedLenFeature(shape=[], dtype='float', default_value=0.0), 'page_views_score': FixedLenFeature(shape=[], dtype='float', default_value=0.0), 'quality_score': FixedLenFeature(shape=[], dtype='float', default_value=0.0), 'name_match': FixedLenFeature(shape=[], dtype='float', default_value=0.0), 'query_text': FixedLenFeature(shape=[], dtype='string', default_value=''), 'domain_id': FixedLenFeature(shape=[], dtype='int64', default_value=0), 'domain_name': FixedLenFeature(shape=[], dtype='string', default_value='')}


INFO:root:Created TFRecordDataset from SequenceExample protobufs from 1 files : ['../ml4ir/applications/ranking/tests/data/csv/tfr
INFO:root:Reading 1 files from [../ml4ir/applications/ranking/tests/data/csv/test/file_0.csv, ..
INFO:root:Writing SequenceExample protobufs to : ../ml4ir/applications/ranking/tests/data/csv/tfrecord/test/file_0.tfrecord


{'query_id': FixedLenFeature(shape=[], dtype='string', default_value=''), 'clicked': FixedLenFeature(shape=[], dtype='int64', default_value=0), 'rank': FixedLenFeature(shape=[], dtype='int64', default_value=0), 'text_match_score': FixedLenFeature(shape=[], dtype='float', default_value=0.0), 'page_views_score': FixedLenFeature(shape=[], dtype='float', default_value=0.0), 'quality_score': FixedLenFeature(shape=[], dtype='float', default_value=0.0), 'name_match': FixedLenFeature(shape=[], dtype='float', default_value=0.0), 'query_text': FixedLenFeature(shape=[], dtype='string', default_value=''), 'domain_id': FixedLenFeature(shape=[], dtype='int64', default_value=0), 'domain_name': FixedLenFeature(shape=[], dtype='string', default_value='')}


INFO:root:Created TFRecordDataset from SequenceExample protobufs from 1 files : ['../ml4ir/applications/ranking/tests/data/csv/tfr


{'query_id': FixedLenFeature(shape=[], dtype='string', default_value=''), 'clicked': FixedLenFeature(shape=[], dtype='int64', default_value=0), 'rank': FixedLenFeature(shape=[], dtype='int64', default_value=0), 'text_match_score': FixedLenFeature(shape=[], dtype='float', default_value=0.0), 'page_views_score': FixedLenFeature(shape=[], dtype='float', default_value=0.0), 'quality_score': FixedLenFeature(shape=[], dtype='float', default_value=0.0), 'name_match': FixedLenFeature(shape=[], dtype='float', default_value=0.0), 'query_text': FixedLenFeature(shape=[], dtype='string', default_value=''), 'domain_id': FixedLenFeature(shape=[], dtype='int64', default_value=0), 'domain_name': FixedLenFeature(shape=[], dtype='string', default_value='')}
<BatchDataset shapes: ({query_id: (128, 1), rank: (128, 1), text_match_score: (128, 1), page_views_score: (128, 1), quality_score: (128, 1), name_match: (128, 1), query_text: (128, 1), domain_id: (128, 1), domain_name: (128, 1)}, (128, 1)), types: ({q

## Let's define a model, already!

### Model Framework

![image.png](images/model_framework.png)

### Step 0: Define the Interaction Model

In [67]:
from ml4ir.base.model.scoring.interaction_model import InteractionModel, UnivariateInteractionModel
from ml4ir.base.config.keys import TFRecordTypeKey
from tensorflow import feature_column

# Define custom feature layer ops
def custom_categorical_embedding(feature_tensor, feature_info, **kwargs):
    """
    Converts input integer tensor into categorical embedding.
    Works by converting the categorical indices in the input feature_tensor,
    represented as integer values, into categorical embeddings based on the feature_info.

    Args:
        feature_tensor: int feature tensor
        feature_info: Dictionary representing the configuration parameters for the specific feature from the FeatureConfig

    Returns:
        categorical embedding for the input feature_tensor

    Args under feature_layer_info:
        num_buckets: int; Maximum number of categorical values
        default_value: int; default value to be assigned to indices out of the num_buckets range
        embedding_size: int; dimension size of the categorical embedding

    NOTE:
    string based categorical features should already be converted into numeric indices
    """
    CATEGORICAL_VARIABLE = "categorical_variable"
    feature_layer_info = feature_info.get("feature_layer_info")

    categorical_fc = feature_column.categorical_column_with_identity(
        CATEGORICAL_VARIABLE,
        num_buckets=feature_layer_info["args"]["num_buckets"],
        default_value=feature_layer_info["args"].get("default_value", None),
    )
    embedding_fc = feature_column.embedding_column(
        categorical_fc, dimension=feature_layer_info["args"]["embedding_size"]
    )

    embedding = layers.DenseFeatures(
        embedding_fc,
        name="{}_embedding".format(feature_info.get("node_name", feature_info["name"])),
    )({CATEGORICAL_VARIABLE: feature_tensor})
    embedding = tf.expand_dims(embedding, axis=1)

    return embedding

feature_layer_fns = {
    "custom_categorical_embedding": custom_categorical_embedding,
}

interaction_model: InteractionModel = UnivariateInteractionModel(
                                            feature_config=feature_config,
                                            feature_layer_keys_to_fns=feature_layer_fns,
                                            tfrecord_type=TFRecordTypeKey.EXAMPLE)

### Step 1: Define the Scorer

In [68]:
from ml4ir.base.model.scoring.scoring_model import ScorerBase, RelevanceScorer
from ml4ir.base.model.losses.loss_base import RelevanceLossBase
from tensorflow.keras import layers
from tensorflow.keras import losses

class MyCustomLoss(RelevanceLossBase):
    def get_loss_fn(self, **kwargs):
        """
        Define a sigmoid cross entropy loss
        Additionally can pass in record positions to handle positional bias

        """
        bce = losses.BinaryCrossentropy(reduction=losses.Reduction.SUM_OVER_BATCH_SIZE)
        mask = kwargs.get("mask")

        def _loss_fn(y_true, y_pred):
            # NOTE: Can use any of the metadata features to qualify your loss here
            return bce(y_true, y_pred)

        return _loss_fn

    def get_final_activation_op(self, output_name):
        return lambda logits, mask: layers.Activation("sigmoid", name=output_name)(logits)

scorer: ScorerBase = RelevanceScorer.from_model_config_file(
    model_config_file='../ml4ir/base/config/default_model_config.yaml',
    interaction_model=interaction_model,
    loss=MyCustomLoss(),
    output_name="relevance_score")
    
logger.info(json.dumps(scorer.model_config, indent=4))

INFO:root:{
    "architecture_key": "dnn",
    "layers": [
        {
            "type": "dense",
            "name": "first_dense",
            "units": 256,
            "activation": "relu"
        },
        {
            "type": "dropout",
            "name": "first_dropout",
            "rate": 0.0
        },
        {
            "type": "dense",
            "name": "second_dense",
            "units": 64,
            "activation": "relu"
        },
        {
            "type": "dropout",
            "name": "second_dropout",
            "rate": 0.0
        },
        {
            "type": "dense",
            "name": "final_dense",
            "units": 1,
            "activation": null
        }
    ]
}


### Step 2: Define Metrics

In [69]:
from tensorflow.keras import metrics as kmetrics

# metrics = ['binary_accuracy', kmetrics.Precision(name='precision')]
metrics = ['binary_accuracy', kmetrics.Precision]

### Step 3: Define Optimizer

In [70]:
from tensorflow.keras.optimizers import Optimizer
from ml4ir.base.model.optimizer import get_optimizer
from ml4ir.base.config.keys import OptimizerKey

optimizer: Optimizer = get_optimizer(
                optimizer_key=OptimizerKey.ADAM,
                learning_rate=0.01,
                learning_rate_decay=0.94,
                learning_rate_decay_steps=1000,
                gradient_clip_value=50,
            )

### Now... let's put it all together

In [71]:
from ml4ir.base.model.relevance_model import RelevanceModel
from ml4ir.base.config.keys import OptimizerKey

relevance_model = RelevanceModel(
        feature_config=feature_config,
        scorer=scorer,
        metrics=metrics,
        optimizer=optimizer,
        tfrecord_type=TFRecordTypeKey.EXAMPLE,
        output_name="relevance_score",
        logger=logger
    )

DEBUG:tensorflow:Transforming feature_column IdentityCategoricalColumn(key='categorical_variable', number_buckets=8, default_value=None).
DEBUG:tensorflow:Transforming feature_column IdentityCategoricalColumn(key='categorical_variable', number_buckets=6, default_value=5).
INFO:root:Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
query_text (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
tf_op_layer_DecodePaddedRaw_6 ( [(None, 1, 20)]      0           query_text[0][0]                 
__________________________________________________________________________________________________
domain_name (InputLayer)        [(None, 1)]          0                                            
_______

In [72]:
if not os.path.exists('../models'):
    os.makedirs('../models')
if not os.path.exists('../logs'):
    os.makedirs('../logs')

relevance_model.fit(relevance_dataset, 
                    num_epochs=5, 
                    models_dir='../models',
                    logs_dir='../logs',
                    monitor_metric='val_binary_accuracy',
                    monitor_mode='max')

INFO:root:Training Model
INFO:root:Starting Epoch : 1
INFO:root:{}


Epoch 1/5


INFO:root:[epoch: 1 | batch: 0] {'batch': 0, 'size': 128, 'loss': 0.6826525, 'binary_accuracy': 0.703125, 'precision_2': 0.35}


      1/Unknown - 7s 7s/step - loss: 0.6827 - binary_accuracy: 0.7031 - precision_2: 0.3500

     25/Unknown - 9s 377ms/step - loss: 0.5766 - binary_accuracy: 0.7347 - precision_2: 0.3500

...

43/44 [============================>.] - ETA: 0s - loss: 0.5497 - binary_accuracy: 0.7353 - precision_2: 0.0000e+00

INFO:root:Evaluating Model
INFO:root:Completed evaluating model
INFO:root:None
INFO:root:End of Epoch 4
INFO:root:{'loss': 0.5499490587548777, 'binary_accuracy': 0.7354403, 'precision_2': 0.0, 'val_loss': 0.5504594336856495, 'val_binary_accuracy': 0.7389915, 'val_precision_2': 0.0}



Epoch 00004: val_binary_accuracy did not improve from 0.73917
Restoring model weights from the end of the best epoch.
44/44 [==============================] - 6s 131ms/step - loss: 0.5499 - binary_accuracy: 0.7354 - precision_2: 0.0000e+00 - val_loss: 0.5505 - val_binary_accuracy: 0.7390 - val_precision_2: 0.0000e+00


INFO:root:Completed training model
INFO:root:None


Epoch 00004: early stopping


### Let's save the model(...and don't forget about serving signatures)

![image.png](images/saved_model.png)

In [73]:
MODEL_DIR = '../models/pointwise_ranking_demo'
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)
    
relevance_model.save(
    models_dir=MODEL_DIR,
    preprocessing_keys_to_fns=preprocessing_fns,
    required_fields_only=True)

DEBUG:tensorflow:Transforming feature_column IdentityCategoricalColumn(key='categorical_variable', number_buckets=6, default_value=5).
DEBUG:tensorflow:Transforming feature_column IdentityCategoricalColumn(key='categorical_variable', number_buckets=8, default_value=None).
DEBUG:tensorflow:Transforming feature_column IdentityCategoricalColumn(key='categorical_variable', number_buckets=6, default_value=5).
DEBUG:tensorflow:Transforming feature_column IdentityCategoricalColumn(key='categorical_variable', number_buckets=8, default_value=None).
DEBUG:tensorflow:Transforming feature_column IdentityCategoricalColumn(key='categorical_variable', number_buckets=6, default_value=5).
DEBUG:tensorflow:Transforming feature_column IdentityCategoricalColumn(key='categorical_variable', number_buckets=8, default_value=None).
DEBUG:tensorflow:Transforming feature_column IdentityCategoricalColumn(key='categorical_variable', number_buckets=6, default_value=5).
DEBUG:tensorflow:Transforming feature_column I

{'rank': FixedLenFeature(shape=[], dtype='int64', default_value=0), 'text_match_score': FixedLenFeature(shape=[], dtype='float', default_value=0.0), 'page_views_score': FixedLenFeature(shape=[], dtype='float', default_value=0.0), 'quality_score': FixedLenFeature(shape=[], dtype='float', default_value=0.0), 'name_match': FixedLenFeature(shape=[], dtype='float', default_value=0.0), 'query_text': FixedLenFeature(shape=[], dtype='string', default_value=''), 'domain_id': FixedLenFeature(shape=[], dtype='int64', default_value=0), 'domain_name': FixedLenFeature(shape=[], dtype='string', default_value='')}


DEBUG:tensorflow:Transforming feature_column IdentityCategoricalColumn(key='categorical_variable', number_buckets=6, default_value=5).
DEBUG:tensorflow:Transforming feature_column IdentityCategoricalColumn(key='categorical_variable', number_buckets=8, default_value=None).
DEBUG:tensorflow:Transforming feature_column IdentityCategoricalColumn(key='categorical_variable', number_buckets=6, default_value=5).
DEBUG:tensorflow:Transforming feature_column IdentityCategoricalColumn(key='categorical_variable', number_buckets=8, default_value=None).
DEBUG:tensorflow:Transforming feature_column IdentityCategoricalColumn(key='categorical_variable', number_buckets=6, default_value=5).
DEBUG:tensorflow:Transforming feature_column IdentityCategoricalColumn(key='categorical_variable', number_buckets=8, default_value=None).
DEBUG:tensorflow:Transforming feature_column IdentityCategoricalColumn(key='categorical_variable', number_buckets=6, default_value=5).
DEBUG:tensorflow:Transforming feature_column I

### Reload the model for some predictions

In [74]:
from ml4ir.base.config.keys import TFRecordTypeKey

relevance_model = RelevanceModel(
    feature_config=feature_config,
    tfrecord_type=TFRecordTypeKey.EXAMPLE,
    model_file=os.path.join(MODEL_DIR, 'final/default/'),
    logger=logger,
    output_name="relevance_score"
)

logger.info("Is Keras model? {}".format(isinstance(relevance_model.model, tf.keras.Model)))
logger.info("Is compiled? {}".format(relevance_model.is_compiled))

relevance_model.predict(test_dataset=relevance_dataset.test).sample(25)

INFO:root:Successfully loaded SavedModel from ../models/pointwise_ranking_demo/final/default/
INFO:root:Is Keras model? True
INFO:root:Is compiled? False
INFO:root:Finished predicting scores for 25 batches


,query_id,clicked,rank,name_match,query_text,domain_name,relevance_score
21,b'query_312',1,1,0.0,b'lbdavmmr04ydc',b'domain_2',0.234175
97,b'query_1029',1,4,0.0,b'76scfjnwz',b'domain_4',0.255646
12,b'query_235',0,2,0.0,b'yp7v0xfio',b'domain_0',0.146192
50,b'query_1081',0,3,0.0,b'pvsvzgydu',b'domain_1',0.460975
26,b'query_727',1,1,0.0,b'5ampmdi9q',b'domain_2',0.236239
55,b'query_743',0,1,1.0,b'u6985gdq',b'domain_3',0.192888
54,b'query_1277',0,5,1.0,b'hu8oqpb',b'domain_2',0.309300
106,b'query_1030',0,3,1.0,b'n8mqju',b'domain_0',0.398232
34,b'query_156',0,3,0.0,b's5v7a5u5nf',b'domain_1',0.245318
106,b'query_1270',1,1,0.0,b'93ppz',b'domain_0',0.212614


### Let's see how the TFRecord serving signature works

In [75]:
from tensorflow.keras import models as kmodels
from tensorflow import data

model = kmodels.load_model(
    os.path.join(MODEL_DIR, 'final/tfrecord/'),
    compile=False)
infer_fn = model.signatures["serving_tfrecord"]

dataset = data.TFRecordDataset(
    glob.glob(os.path.join(CSV_DATA_DIR, "tfrecord", "test", "*.tfrecord")))
protos = next(iter(dataset.batch(10)))

logger.info("Example proto: \n{}".format(protos[0]))

logger.info("\n\n\nPredictions:")
logger.info(infer_fn(protos=protos))

INFO:root:Example proto: 
b'\n\xf7\x01\n\x1c\n\x10page_views_score\x12\x08\x12\x06\n\x04]\x04G>\n\x16\n\nname_match\x12\x08\x12\x06\n\x04\x00\x00\x80?\n\x19\n\rquality_score\x12\x08\x12\x06\n\x04\x9b \x9a>\n\x17\n\x08query_id\x12\x0b\n\t\n\x07query_1\n\x1b\n\x0bdomain_name\x12\x0c\n\n\n\x08domain_1\n\r\n\x04rank\x12\x05\x1a\x03\n\x01\x02\n\x1b\n\nquery_text\x12\r\n\x0b\n\tLVA3934GV\n\x10\n\x07clicked\x12\x05\x1a\x03\n\x01\x00\n\x1c\n\x10text_match_score\x12\x08\x12\x06\n\x044\xb4\x19?\n\x12\n\tdomain_id\x12\x05\x1a\x03\n\x01\x01'
INFO:root:


Predictions:
INFO:root:{'relevance_score': <tf.Tensor: id=469044, shape=(10, 1), dtype=float32, numpy=
array([[0.42908183],
       [0.46207318],
       [0.24165526],
       [0.42101735],
       [0.42176935],
       [0.23088714],
       [0.42101735],
       [0.46448255],
       [0.46483183],
       [0.46320197]], dtype=float32)>}


### Why you should onboard your ML application to ml4ir today!

* Consistent code structure and modularization across projects
* Scalable TFRecord data pipeline
    * Every ML application shouldn’t have to reinvent the wheel especially when there is barely any documentation on this.
    * Consistent file I/O overall
* Consistent library versions across projects
    * Easily update versions and validate inference time impact, etc
* Common Flowsnake enablement
    * We can define _git.soma/MLConfigs_ to track and automatically build docker images through strata from ml4ir.
* Unified python ↔ JVM interoperability
    * Define integration tests
    * Allows us to build generic protobuf creation at runtime
* Common training abstraction
    * Callbacks : checkpointing, early stopping, tensorboard, etc
    * Consistent way to save models
        * allows us to have generic deployment code
* Shared metrics, losses, layers, etc.
* Shared feature processing and feature layers across ML models
    * long term: shared NLP toolkit, probability toolkit
    * short term: categorical, text embeddings
* Build models that can be trained with tight coupling:
    * transfer learning
    * shared embedding layers
    * multi task models



> ### This is just the `end of the beginning` and we would love to take new passengers on this journey!

![thanks](images/thats_all_folks.gif)

                                                            .


                                                            .


                                                            .


                                                            .


                                                            .


                                                            .


                                                            .


                                                            .

<center>Psst... You can file github issues -> <a href="https://github.com/salesforce/ml4ir/issues">HERE!</a></center>

![shhh](images/chris_evans_shush.gif)